In [29]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import *
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.impute import SimpleImputer
#import graphviz 
from sklearn import tree
#import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier

In [8]:
train=pd.read_csv(r"/home/acts1/dbdag4/airplane/3c055e822d5b11ea/train.csv")
test=pd.read_csv(r"/home/acts1/dbdag4/airplane/3c055e822d5b11ea/test.csv")
train.isnull().sum()

print(train.Severity.head())
print(definitions)

X_temp=train.drop(['Severity','Accident_ID'],axis=1)
y=train['Severity']
y.unique()

scaler = StandardScaler()
X_scale=scaler.fit_transform(X_temp)
X=pd.DataFrame(X_scale)
X.columns=X_temp.columns


####test

test.isnull().sum()


X_temp_test=test.drop(['Accident_ID'],axis=1)

scaler = StandardScaler()
X_scale_test=scaler.fit_transform(X_temp_test)
X_test=pd.DataFrame(X_scale_test)
X_test.columns=X_temp_test.columns

kfold = KFold(n_splits=10, random_state=17077)

0                  Minor_Damage_And_Injuries
1                  Minor_Damage_And_Injuries
2          Significant_Damage_And_Fatalities
3    Significant_Damage_And_Serious_Injuries
4          Significant_Damage_And_Fatalities
Name: Severity, dtype: object
Index(['Minor_Damage_And_Injuries', 'Significant_Damage_And_Fatalities',
       'Significant_Damage_And_Serious_Injuries', 'Highly_Fatal_And_Damaging'],
      dtype='object')


/home/acts1/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [9]:
lr_range = [0.001, 0.01, 0.1, 0.2,0.25, 0.3]
n_est_range = [10,20,30,50,100]
md_range = [2,4,6,8,10]

parameters = dict(learning_rate=lr_range,
                  n_estimators=n_est_range,
                  max_depth=md_range)

clf = XGBClassifier(random_state=2000)
cv_xg = GridSearchCV(clf, param_grid=parameters,
                  cv=kfold,n_jobs=-1,scoring='accuracy')

cv_xg.fit(X,y)

#y_pred=cv_xg.predict_proba(X_test)[:,1]

print(cv_xg.best_params_)

print(cv_xg.best_score_)


{'learning_rate': 0.3, 'max_depth': 8, 'n_estimators': 100}
0.9659000000000001


In [6]:
X_test

,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric
0,-1.371727,0.866845,-0.080077,0.592957,0.109134,-0.616620,0.077985,0.586995,0.009034,-0.467493
1,1.004384,0.582969,-0.498014,-0.068431,-1.071998,-0.513424,1.676045,-0.230758,0.009034,-0.640138
2,-0.524519,0.582969,-0.498014,-0.056478,-0.338031,2.567706,1.143358,-1.561731,-0.972910,-0.636168
3,-2.367618,2.286227,-0.219389,0.086956,0.389769,0.319513,-0.454701,0.825254,0.009034,-0.131314
4,-1.901934,1.434598,-0.637327,-0.809504,-0.538484,-0.284919,-0.987388,0.328201,0.009034,0.600666
...,...,...,...,...,...,...,...,...,...,...
2495,1.615945,-1.404167,0.616486,-1.371285,0.809178,-0.808269,-0.987388,1.673323,0.009034,1.053963
2496,0.799595,-1.120290,0.337861,0.509288,0.349678,-0.082213,1.143358,-1.225446,-0.972910,-0.634379
2497,-0.314119,0.299092,0.477173,2.258380,-0.532316,1.462037,1.143358,0.452692,0.009034,-0.625041
2498,-0.190685,0.015216,1.034423,0.680611,-0.865377,-0.963062,1.143358,-0.861711,-0.972910,-0.631188


In [10]:
#################################random forest#################################

parameters = {'max_features': np.arange(1,11)}
model_rf = RandomForestClassifier(random_state=1211)
cv_rf = GridSearchCV(model_rf, param_grid=parameters,cv=kfold,n_jobs=-1,scoring='accuracy',verbose=3)

cv_rf.fit( X , y )
print(cv_rf.best_params_)
print(cv_rf.best_score_)



Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 28 concurrent workers.
[Parallel(n_jobs=-1)]: Done  79 out of 100 | elapsed:    6.2s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    8.6s finished


{'max_features': 7}
0.9561


In [23]:
y_pred_rf=cv_rf.predict(X_test)
output_rf=pd.DataFrame()
output_rf['Accident_ID']=test['Accident_ID']
output_rf['Severity']=y_pred_rf
output_rf.to_csv(r"output_rf.csv",index=False)

In [11]:
#########################mlp####################################


lr_range = np.linspace(0.01,0.8,10)
hl_range = [(4,3,2),(3,2),(5,4,3,2),(10,5,3)]
act_range = ['logistic','tanh','relu','identity']

parameters = dict(learning_rate_init=lr_range,hidden_layer_sizes = hl_range,
                  activation = act_range)

mlp = MLPClassifier(random_state=17077,learning_rate='constant')
mlpGrid = GridSearchCV(mlp, param_grid=parameters, cv=kfold,scoring='accuracy',n_jobs=-1,verbose=3)
mlpGrid.fit(X, y)

print(mlpGrid.best_params_)

print(mlpGrid.best_score_)

#{'activation': 'tanh', 'hidden_layer_sizes': (10, 5, 3), 'learning_rate_init': 0.01}
# 0.9397





Fitting 10 folds for each of 160 candidates, totalling 1600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 28 concurrent workers.
[Parallel(n_jobs=-1)]: Done  72 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 232 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 456 tasks      | elapsed:   19.1s
[Parallel(n_jobs=-1)]: Done 744 tasks      | elapsed:   28.3s
[Parallel(n_jobs=-1)]: Done 1096 tasks      | elapsed:   37.6s
[Parallel(n_jobs=-1)]: Done 1512 tasks      | elapsed:   45.4s
[Parallel(n_jobs=-1)]: Done 1600 out of 1600 | elapsed:   46.7s finished


{'activation': 'tanh', 'hidden_layer_sizes': (10, 5, 3), 'learning_rate_init': 0.01}
0.9414999999999999


In [6]:
############################decision tree#############################
depth_range = [3,5,14]
minsplit_range =[3,5,7,9]
minleaf_range =[3,5,7]
##14/7/3
parameters = dict(max_depth=depth_range,min_samples_split=minsplit_range,
                  min_samples_leaf=minleaf_range)

kfold = StratifiedKFold(n_splits=5, random_state=17077)

clf = DecisionTreeClassifier(random_state=17077)
cv_dt = GridSearchCV(clf, param_grid=parameters,cv=kfold,scoring='accuracy',n_jobs=-1,verbose=3)
cv_dt.fit(X,y)

print(cv_dt.best_params_)

print(cv_dt.best_score_)
#########################################################


Fitting 5 folds for each of 36 candidates, totalling 180 fits


/home/acts1/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 28 concurrent workers.


{'max_depth': 14, 'min_samples_leaf': 3, 'min_samples_split': 7}
0.9259999999999999


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    0.5s finished


In [12]:
############################lda##########################


lda = LinearDiscriminantAnalysis()
cv_lda = cross_val_score(lda, X, y, cv=kfold,scoring='accuracy',verbose=3)
cv_lda.mean()
#########################################################



[CV]  ................................................................
[CV] .................................... , score=0.627, total=   0.2s


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV]  ................................................................
[CV] .................................... , score=0.641, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.654, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.631, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.644, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.659, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.660, total=   0.0s
[CV]  ................................................................
[CV] .................................... , score=0.647, total=   0.0s
[CV]  

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.4s finished


0.6463000000000001

In [13]:
##########################qda###############################


qda = QuadraticDiscriminantAnalysis()
cv_qda = cross_val_score(qda, X, y, cv=kfold,n_jobs=-1,scoring='accuracy',verbose=3)
cv_qda.mean()
#########################################################


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 28 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished


0.7946000000000001

In [14]:
############################logisitc#############################

logreg = LogisticRegression()
cv_logreg = cross_val_score(logreg, X, y, cv=kfold,n_jobs=-1,scoring='accuracy',verbose=3)
cv_logreg.mean()
#########################################################


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 28 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.1s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.2s finished


0.6408999999999999

In [15]:
########################stacking

model_rf = RandomForestClassifier(random_state=1211)
model_mlp = MLPClassifier(random_state=17077,learning_rate='constant')
ourEstimators = [('RF',model_rf),('MNP',model_mlp)]
xgb = XGBClassifier()
reg = StackingClassifier(estimators=ourEstimators,cv=kfold,final_estimator=xgb,passthrough=False)
#reg.fit(X, y)


cv = cross_val_score(reg, X,y,cv=kfold,n_jobs=-1,verbose=3)
cv.mean()

# 0.9518000000000001


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 28 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:  1.1min remaining:  2.7min
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:  1.2min remaining:   29.7s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.2min finished


0.9552999999999999

In [16]:
### or
ourEstimators = [('XGB',xgb),('MNP',model_mlp)]
reg = StackingClassifier(estimators=ourEstimators,cv=kfold,final_estimator=model_rf,passthrough=False)
cv_1 = cross_val_score(reg, X,y,cv=kfold,n_jobs=-1,verbose=3)
cv_1.mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 28 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:  1.3min remaining:  3.1min
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:  1.3min remaining:   34.7s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.4min finished


0.9469999999999998

In [17]:
### or
ourEstimators = [('XGB',xgb),('RF',model_rf),('MNP',model_mlp)]
reg = StackingClassifier(estimators=ourEstimators,cv=kfold,final_estimator=xgb,passthrough=False)
cv_2 = cross_val_score(reg, X,y,cv=kfold,n_jobs=-1,verbose=3)
cv_2.mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 28 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:  1.5min remaining:  3.5min
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:  1.6min remaining:   40.5s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.6min finished


0.9560999999999998

In [18]:
### or
ourEstimators = [('RF',model_rf)]
reg = StackingClassifier(estimators=ourEstimators,cv=kfold,final_estimator=xgb,passthrough=False)
cv_3 = cross_val_score(reg, X,y,cv=kfold,n_jobs=-1,verbose=3)
cv_3.mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 28 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:   13.7s remaining:   32.0s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:   13.9s remaining:    5.9s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   13.9s finished


0.9382999999999999

In [19]:
### or
ourEstimators = [('XGB',xgb)]
reg = StackingClassifier(estimators=ourEstimators,cv=kfold,final_estimator=model_rf,passthrough=False)
cv_4 = cross_val_score(reg, X,y,cv=kfold,n_jobs=-1,verbose=3)
cv_4.mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 28 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:   15.9s remaining:   37.2s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:   16.0s remaining:    6.8s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   16.1s finished


0.9023999999999999

In [24]:
model_dt = DecisionTreeClassifier()
model_lda = LinearDiscriminantAnalysis()
model_qda = QuadraticDiscriminantAnalysis()
model_logreg = LogisticRegression()
ourEstimators = [('RF',model_rf),('MNP',model_mlp),('DT',model_dt),('LDA',model_lda),('QDA',model_qda),
                 ('logreg',model_logreg)]
reg_s6 = StackingClassifier(estimators=ourEstimators,cv=kfold,final_estimator=xgb,passthrough=False)
cv_5 = cross_val_score(reg_s6, X,y,cv=kfold,n_jobs=-1,verbose=3)
cv_5.mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 28 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:  1.3min remaining:  3.1min
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:  1.4min remaining:   36.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.4min finished


0.9577

In [27]:
reg_s6.fit(X,y)

/home/acts1/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/acts1/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/acts1/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/acts1/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't conver

StackingClassifier(cv=KFold(n_splits=10, random_state=17077, shuffle=False),
                   estimators=[('RF',
                                RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       max_samples=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_sampl

In [28]:
y_pred_s6=reg_s6.predict(X_test)
output_s6=pd.DataFrame()
output_s6['Accident_ID']=test['Accident_ID']
output_s6['Severity']=y_pred_s6
output_s6.to_csv(r"output_s6.csv",index=False)

In [21]:
ourEstimators = [('MNP',model_mlp),('DT',model_dt),('LDA',model_lda),('QDA',model_qda),
                 ('logreg',model_logreg)]
finalEstimators = [('RF',model_rf),('XGB',xgb)]
reg = StackingClassifier(estimators=ourEstimators,cv=kfold,final_estimator=finalEstimators,passthrough=False)
cv_6 = cross_val_score(reg, X,y,cv=kfold,n_jobs=-1,verbose=3)
cv_6.mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 28 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished


nan

In [35]:
#voting=VotingClassifier(estimators=[('RF',model_rf),('XGB',xgb)],weights=np.array([0.3,0.5,0.2]))
voting=VotingClassifier(estimators=[('RF',model_rf),('XGB',xgb)])
voting.fit(X,y)

VotingClassifier(estimators=[('RF',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
        

In [36]:
y_pred_v=voting.predict(X_test)
output_v=pd.DataFrame()
output_v['Accident_ID']=test['Accident_ID']
output_v['Severity']=y_pred_v
output_v.to_csv(r"output_v2.csv",index=False)

In [17]:
print("xg")
print(cv_xg.best_score_)
print("rf")
print(cv_rf.best_score_)
print("mlp")
print(mlpGrid.best_score_)
print("dt")
print(cv_dt.best_score_)
print("lda")
print(cv_lda.mean())
print("qda")
print(cv_qda.mean())
print("logreg")
print(cv_logreg.mean())
print("stack1")
print(cv.mean())
print("stack2")
print(cv_1.mean())
print("stack3")
print(cv_2.mean())
print("stack4")
print(cv_3.mean())
print("stack5")
print(cv_4.mean())
print("stack6")
print(cv_5.mean())
print("stack7")
print(cv_6.mean())

xg
0.9659000000000001
rf
0.9560000000000001
mlp
0.9447000000000001
dt
0.9259999999999999
lda
0.6463
qda
0.7938000000000001
logreg
0.6391
stack1
0.9536999999999999
stack2
0.9443999999999999
stack3
0.9529
stack4
0.9342
stack5
0.8992000000000001
stack6
0.9559
stack7
nan
